In [1]:
!pip install pycocotools

In [2]:
from pycocotools.coco import COCO
import os
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt

from yacs.config import CfgNode as CN 
from glob import glob
from natsort import natsorted
import numpy as np
import h5py

# for volume stiching
from cellpose import models, plot, io, metrics

%matplotlib inline

2022-03-01 15:53:13,194 [INFO] WRITING LOG OUTPUT TO /n/home05/lauenburg/.cellpose/run.log


In [3]:
_C = CN()

# path to the image folder
#_C.IMG_ROOT = "/n/pfister_lab2/Lab/leander/cerberus/img_toolbox/CVAT_vol0_full_v1_merged2_rgb/*.jpeg"
#_C.IMG_ROOT = "/n/pfister_lab2/Lab/leander/cerberus/img_toolbox/CVAY_vol0_full_v1_merged2_CVAT1_rgb/*.jpeg"
_C.IMG_ROOT = "/n/pfister_lab2/Lab/leander/cerberus/img_toolbox/CVAY_vol4/*.jpeg"

# path to the coco json file
#_C.COCO_JSON = "/n/pfister_lab2/Lab/leander/cerberus/img_toolbox/CVAT_vol0_full_v1_merged2_rgb/CVAT_vol0_full_v1_merged2_rgb.json"
#_C.COCO_JSON = "/n/pfister_lab2/Lab/leander/cerberus/img_toolbox/CVAY_vol0_full_v1_merged2_CVAT1_rgb/CVAY_vol0_full_v1_merged2_CVAT1_rgb.json"
_C.COCO_JSON = "/n/pfister_lab2/Lab/leander/cerberus/img_toolbox/CVAY_vol4/CVAY_vol4_second.json"

# path for label target dir
#_C.TARGET_DIR = "/n/pfister_lab2/Lab/leander/cerberus/img_toolbox/CVAT_vol0_full_v1_merged2_rgb/labels"
#_C.TARGET_DIR = "/n/pfister_lab2/Lab/leander/cerberus/img_toolbox/CVAY_vol0_full_v1_merged2_CVAT1_rgb/labels"
_C.TARGET_DIR = "/n/pfister_lab2/Lab/leander/cerberus/img_toolbox/CVAY_vol4/labels"


In [4]:
# load the configurations
def get_cfg_defaults():
    return _C.clone()

cfg = get_cfg_defaults()

# Generate 2D masks from coco json

In [5]:
# load the json
coco = COCO(cfg.COCO_JSON)
# load the paths to all imager
image_paths = natsorted(glob(cfg.IMG_ROOT))
print(len(image_paths))

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
255


In [6]:
# create the masks
anns_list = []
for i, img_p in enumerate(image_paths):
    img = coco.imgs[i+1]
    cat_ids = coco.getCatIds()
    anns_ids = coco.getAnnIds(imgIds=img['id'], catIds=cat_ids, iscrowd=None)
    anns = coco.loadAnns(anns_ids)
    anns_img = np.zeros((img['height'],img['width']))
    if len(anns) > 0:
        anns_img = np.maximum(anns_img,coco.annToMask(anns[0])*anns[0]['category_id'])
        for ann in anns[1:]:
            anns_img += np.maximum(anns_img,coco.annToMask(ann)*ann['category_id'])
    anns_list.append(anns_img)

In [7]:
anns_list = np.asarray(anns_list).astype(np.uint16)

In [8]:
assert(np.asarray(anns_list).shape == (255, 2048, 2048)), f"The list has shape of {np.asarray(anns_list).shape} but should be (255, 2048, 2048)"

In [9]:
# Generated 3D masks from 2D masks

In [10]:
def stitch3D(masks, stitch_threshold=0.25):
    """ stitch 2D masks into 3D volume with stitch_threshold on IOU """
    
    mmax = masks[0].max()
    empty = 0
    
    for i in range(len(masks)-1):
        iou = metrics._intersection_over_union(masks[i+1], masks[i])[1:,1:]
        if not iou.size and empty == 0:
            masks[i+1] = masks[i+1]
            mmax = masks[i+1].max()
        elif not iou.size and not empty == 0:
            icount = masks[i+1].max()
            istitch = np.arange(mmax+1, mmax + icount+1, 1, int)
            mmax += icount
            istitch = np.append(np.array(0), istitch)
            masks[i+1] = istitch[masks[i+1]]
        else:
            iou[iou < stitch_threshold] = 0.0
            iou[iou < iou.max(axis=0)] = 0.0
            istitch = iou.argmax(axis=1) + 1
            ino = np.nonzero(iou.max(axis=1)==0.0)[0]
            istitch[ino] = np.arange(mmax+1, mmax+len(ino)+1, 1, int)
            mmax += len(ino)
            istitch = np.append(np.array(0), istitch)
            masks[i+1] = istitch[masks[i+1]]
            empty = 1
            
    return masks

In [11]:
stitched_volume = stitch3D(np.asarray(anns_list))

/n/home05/lauenburg/.conda/envs/img_toolbox/lib/python3.7/site-packages/cellpose/metrics.py:202: RuntimeWarning: invalid value encountered in true_divide
  iou = overlap / (n_pixels_pred + n_pixels_true - overlap)


In [12]:
import sys
assert(np.asarray(stitched_volume).shape == (255, 2048, 2048)), f"The list has shape of {np.asarray(anns_list).shape} but should be (255, 2048, 2048)"
assert(stitched_volume.dtype in ["uint16", "uint32", "uint64"]), "The input label volume should be of type uint8 | uint16 | uint32 | uint64"
print(stitched_volume.dtype)
print(sys.getsizeof(stitched_volume))


uint16
2139095176


In [13]:
with h5py.File(os.path.join(cfg.TARGET_DIR, "CVAY_vol4_second.h5"), "w") as h5:
    h5_dataset =  h5.create_dataset("main", data=stitched_volume)

In [14]:
print(os.path.join(cfg.TARGET_DIR, "CVAY_vol4_second.h5"))

/n/pfister_lab2/Lab/leander/cerberus/img_toolbox/CVAY_vol4/labels/CVAY_vol4_second.h5
